In [2]:
#recent_matches
import requests
import pymysql
from datetime import datetime

# ---------------- EXTRACT ----------------
def fetch_api(url, headers=None):
    if headers is None:
        headers = {
            "x-rapidapi-key": "e6f772e53cmsh35b97b5b358e839p17d820jsn89d15a173062",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# ---------------- TRANSFORM ----------------
def format_scores(team_score):
    if not team_score:
        return ""
    scores = []
    for _, innings in team_score.items():
        if innings:
            runs = innings.get("runs", 0)
            wickets = innings.get("wickets", 0)
            overs = innings.get("overs", 0)
            scores.append(f"{runs}/{wickets} ({overs} ov)")
    return " | ".join(scores)

def extract_recent_matches(data):
    all_matches = []
    type_matches = data.get("typeMatches", [])

    for t in type_matches:
        series_matches = t.get("seriesMatches", [])
        for s in series_matches:
            series_ad_wrapper = s.get("seriesAdWrapper", {})
            matches = series_ad_wrapper.get("matches", [])
            for m in matches:
                info = m.get("matchInfo", {})
                score = m.get("matchScore", {})

                team1_score = format_scores(score.get("team1Score"))
                team2_score = format_scores(score.get("team2Score"))

                start_dt = datetime.fromtimestamp(int(info.get("startDate", 0)) / 1000)

                match_data = {
                    "match_id": info.get("matchId"),
                    "match_desc": info.get("matchDesc"),
                    "team1": info.get("team1", {}).get("teamName"),
                    "team2": info.get("team2", {}).get("teamName"),
                    "start_date": start_dt.strftime("%Y-%m-%d %H:%M:%S"),
                    "status": info.get("status"),
                    "state": info.get("state"),
                    "venue": info.get("venueInfo", {}).get("ground"),
                    "venue_city": info.get("venueInfo", {}).get("city"),
                    "series_name": info.get("seriesName"),
                    "match_format": info.get("matchFormat"),
                    "team1_score": team1_score,
                    "team2_score": team2_score
                }
                all_matches.append(match_data)
    return all_matches

# ---------------- LOAD ----------------
def create_table_if_not_exists(conn):
    create_query = """
    CREATE TABLE IF NOT EXISTS recent_matches (
        match_id BIGINT PRIMARY KEY,
        match_desc VARCHAR(100),
        team1 VARCHAR(50),
        team2 VARCHAR(50),
        start_date DATETIME,
        status VARCHAR(100),
        state VARCHAR(50),
        venue VARCHAR(100),
        venue_city VARCHAR(100),
        series_name VARCHAR(150),
        match_format VARCHAR(20),
        team1_score VARCHAR(50),
        team2_score VARCHAR(50)
    )
    """
    with conn.cursor() as cursor:
        cursor.execute(create_query)
    conn.commit()

def insert_matches(conn, matches):
    insert_query = """
    REPLACE INTO recent_matches 
    (match_id, match_desc, team1, team2, start_date, status, state, venue, venue_city,
     series_name, match_format, team1_score, team2_score)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = [
        (
            m["match_id"], m["match_desc"], m["team1"], m["team2"], m["start_date"],
            m["status"], m["state"], m["venue"], m["venue_city"], m["series_name"],
            m["match_format"], m["team1_score"], m["team2_score"]
        )
        for m in matches
    ]
    with conn.cursor() as cursor:
        cursor.executemany(insert_query, values)
    conn.commit()
    print(f"✅ {len(matches)} matches inserted/updated successfully.")

# ---------------- MAIN ETL ----------------
if __name__ == "__main__":
    # Extract
    url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
    data = fetch_api(url)

    # Transform
    matches = extract_recent_matches(data)

    # Load
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api",
        autocommit=False
    )
    create_table_if_not_exists(conn)
    insert_matches(conn, matches)
    conn.close()


✅ 52 matches inserted/updated successfully.


In [3]:
#players
import requests
import pymysql
import pandas as pd

# ------------------ DB CONNECTION ------------------
def get_connection():
    return pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api",
        autocommit=False
    )

# ------------------ GENERIC FETCH ------------------
def fetch_api(url, headers=None):
    if headers is None:
        headers = {
            "x-rapidapi-key": "e6f772e53cmsh35b97b5b358e839p17d820jsn89d15a173062",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# ------------------ TRANSFORM HELPERS ------------------
def extract_stat(data, stat_name, format_type):
    headers = data.get("headers", [])
    values_list = data.get("values", [])
    
    if format_type not in headers:
        return 0
    col_index = headers.index(format_type)
    
    for item in values_list:
        row = item.get("values", [])
        if row and row[0].lower() == stat_name.lower():
            try:
                return int(row[col_index].replace(",", ""))
            except Exception:
                return 0
    return 0

def sum_stats_across_formats(data, stat_name):
    formats = ["Test", "ODI", "T20"]
    return sum(extract_stat(data, stat_name, fmt) for fmt in formats)

# ------------------ PLAYERS PIPELINE ------------------
def fetch_players_summary(player_ids):
    players = []
    for pid in player_ids:
        try:
            info = fetch_api(f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}")
            bat = fetch_api(f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}/batting")
            bowl = fetch_api(f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}/bowling")
            
            players.append({
                "player_id": pid,
                "full_name": info.get("name"),
                "name": info.get("nickName"),
                "country": info.get("intlTeam"),
                "playing_role": info.get("role"),
                "batting_style": info.get("bat"),
                "bowling_style": info.get("bowl"),
                "total_runs": sum_stats_across_formats(bat, "Runs"),
                "total_wickets": sum_stats_across_formats(bowl, "Wickets")
            })
            print(f"✅ Player {pid} fetched")
        except Exception as e:
            print(f"⚠️ Error fetching player {pid}: {e}")
    return players

def create_players_table(conn):
    create_query = """
    CREATE TABLE IF NOT EXISTS players (
        player_id INT PRIMARY KEY,
        full_name VARCHAR(100),
        name VARCHAR(150),
        country VARCHAR(50),
        playing_role VARCHAR(50),
        batting_style VARCHAR(50),
        bowling_style VARCHAR(50),
        total_runs INT,
        total_wickets INT
    )
    """
    with conn.cursor() as cursor:
        cursor.execute(create_query)
    conn.commit()

def insert_players(conn, df):
    insert_query = """
    REPLACE INTO players
    (player_id, full_name, name, country, playing_role, batting_style, bowling_style, total_runs, total_wickets)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    values = [
        (int(r["player_id"]), r["full_name"], r["name"], r["country"], r["playing_role"],
         r["batting_style"], r["bowling_style"], int(r["total_runs"]), int(r["total_wickets"]))
        for _, r in df.iterrows()
    ]
    with conn.cursor() as cursor:
        cursor.executemany(insert_query, values)
    conn.commit()
    print(f"✅ {len(values)} players inserted/updated.")

# ------------------ MAIN ETL ------------------
if __name__ == "__main__":
    conn = get_connection()

    # Define player IDs you want
    player_ids = [
        576, 25, 265, 1413, 11808, 8733, 10808, 13217, 10551, 9311, 587, 8808, 9647, 10276, 10744, 8271,11540, 18504, 9720, 10738, 9551, 11177,
               10100, 8313
    ]

    # Extract + Transform
    players_summary = fetch_players_summary(player_ids)
    df_players = pd.DataFrame(players_summary)

    # Load
    create_players_table(conn)
    insert_players(conn, df_players)

    conn.close()
    print("🎉 Players ETL pipeline completed successfully.")


✅ Player 576 fetched
✅ Player 25 fetched
✅ Player 265 fetched
✅ Player 1413 fetched
✅ Player 11808 fetched
✅ Player 8733 fetched
✅ Player 10808 fetched
✅ Player 13217 fetched
✅ Player 10551 fetched
✅ Player 9311 fetched
✅ Player 587 fetched
✅ Player 8808 fetched
✅ Player 9647 fetched
✅ Player 10276 fetched
✅ Player 10744 fetched
✅ Player 8271 fetched
✅ Player 11540 fetched
✅ Player 18504 fetched
✅ Player 9720 fetched
✅ Player 10738 fetched
✅ Player 9551 fetched
✅ Player 11177 fetched
✅ Player 10100 fetched
✅ Player 8313 fetched
✅ 24 players inserted/updated.
🎉 Players ETL pipeline completed successfully.


In [7]:
#series
import requests
import pymysql
from datetime import datetime

# ----------------- EXTRACT -----------------
def fetch_api(url, headers=None):
    if headers is None:
        headers = {
            "x-rapidapi-key": "0ef749c92cmshb8e26740cefe8c1p143f2bjsn5d2ede97ace6",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# ----------------- TRANSFORM -----------------
def format_scores(team_score):
    if not team_score:
        return ""
    scores = []
    for _, innings in team_score.items():
        if innings:
            runs = innings.get("runs", 0)
            wickets = innings.get("wickets", 0)
            overs = innings.get("overs", 0)
            scores.append(f"{runs}/{wickets} ({overs} ov)")
    return " | ".join(scores)

def extract_series_matches(data):
    all_matches = []
    match_details = data.get("matchDetails", [])

    for item in match_details:
        map_data = item.get("matchDetailsMap")
        if not map_data:
            continue

        matches = map_data.get("match", [])
        for m in matches:
            info = m.get("matchInfo", {})
            score = m.get("matchScore", {})

            team1_score = format_scores(score.get("team1Score"))
            team2_score = format_scores(score.get("team2Score"))

            start_dt = datetime.fromtimestamp(int(info.get("startDate", 0)) / 1000)
            end_dt = datetime.fromtimestamp(int(info.get("endDate", 0)) / 1000)

            match_data = {
                "match_id": info.get("matchId"),
                "match_desc": info.get("matchDesc"),
                "team1": info.get("team1", {}).get("teamName"),
                "team2": info.get("team2", {}).get("teamName"),
                "start_date": start_dt,
                "end_date": end_dt,
                "status": info.get("status"),
                "state": info.get("state"),
                "venue": info.get("venueInfo", {}).get("ground"),
                "series_name": info.get("seriesName"),
                "match_format": info.get("matchFormat"),
                "team1_score": team1_score,
                "team2_score": team2_score
            }
            all_matches.append(match_data)

    return pd.DataFrame(all_matches)

# ----------------- LOAD -----------------
def get_connection():
    return pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api",
        autocommit=False
    )

def create_series_matches_table(conn):
    create_query = """
    CREATE TABLE IF NOT EXISTS series_matches (
        match_id BIGINT PRIMARY KEY,
        match_desc VARCHAR(100),
        team1 VARCHAR(50),
        team2 VARCHAR(50),
        start_date DATETIME,
        end_date DATETIME,
        status VARCHAR(100),
        state VARCHAR(50),
        venue VARCHAR(100),
        series_name VARCHAR(150),
        match_format VARCHAR(20),
        team1_score VARCHAR(50),
        team2_score VARCHAR(50)
    )
    """
    with conn.cursor() as cursor:
        cursor.execute(create_query)
    conn.commit()

def insert_series_matches(conn, df):
    insert_query = """
    REPLACE INTO series_matches
    (match_id, match_desc, team1, team2, start_date, end_date, status, state,
     venue, series_name, match_format, team1_score, team2_score)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    values = [
        (
            int(r["match_id"]), r["match_desc"], r["team1"], r["team2"],
            r["start_date"], r["end_date"], r["status"], r["state"],
            r["venue"], r["series_name"], r["match_format"],
            r["team1_score"], r["team2_score"]
        )
        for _, r in df.iterrows()
    ]
    with conn.cursor() as cursor:
        cursor.executemany(insert_query, values)
    conn.commit()
    print(f"✅ {len(values)} matches inserted/updated.")

# ----------------- MAIN ETL -----------------
if __name__ == "__main__":
    # Series IDs to process
    series_ids = [8393, 4905, 8528, 3692]

    # 1. Extract
    all_matches_df = pd.DataFrame()
    for sid in series_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{sid}"
        data = fetch_api(url)
        df = extract_series_matches(data)
        all_matches_df = pd.concat([all_matches_df, df], ignore_index=True)

    # 2. Load
    conn = get_connection()
    create_series_matches_table(conn)
    insert_series_matches(conn, all_matches_df)
    conn.close()

    print("🎉 Series Matches ETL completed successfully.")


✅ 21 matches inserted/updated.
🎉 Series Matches ETL completed successfully.


In [6]:
#top_odi_score
import requests
import pymysql

# ----------------- EXTRACT -----------------
def fetch_top_odi_runs(api_key):
    """
    Fetches top ODI run scorers from Cricbuzz API and returns a DataFrame.
    """
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"
    querystring = {"statsType": "mostRuns", "matchType": "2"}  # "2" = ODI
    
    headers = {
        "x-rapidapi-key": "0ef749c92cmshb8e26740cefe8c1p143f2bjsn5d2ede97ace6",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status()
    data = response.json()
    
    # ----------------- TRANSFORM -----------------
    rows = []
    values = data.get("values", [])
    
    for item in values:
        row = item.get("values", [])
        if len(row) >= 6:
            try:
                rows.append({
                    "player_id": int(row[0]),
                    "player_name": row[1],
                    "matches": int(row[2]),
                    "innings": int(row[3]),
                    "runs": int(row[4]),
                    "average": float(row[5])
                })
            except ValueError:
                continue
    
    return pd.DataFrame(rows)

# ----------------- LOAD -----------------
def get_connection():
    return pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api",
        autocommit=False
    )

def create_top_odi_runs_table(conn):
    create_query = """
    CREATE TABLE IF NOT EXISTS top_odi_runs (
        player_id BIGINT PRIMARY KEY,
        player_name VARCHAR(100),
        matches INT,
        innings INT,
        runs INT,
        average FLOAT
    )
    """
    with conn.cursor() as cursor:
        cursor.execute(create_query)
    conn.commit()

def insert_top_odi_runs(conn, df):
    insert_query = """
    REPLACE INTO top_odi_runs
    (player_id, player_name, matches, innings, runs, average)
    VALUES (%s,%s,%s,%s,%s,%s)
    """
    values = [
        (int(r["player_id"]), r["player_name"], int(r["matches"]),
         int(r["innings"]), int(r["runs"]), float(r["average"]))
        for _, r in df.iterrows()
    ]
    with conn.cursor() as cursor:
        cursor.executemany(insert_query, values)
    conn.commit()
    print(f"✅ {len(values)} ODI records inserted/updated.")

# ----------------- MAIN ETL -----------------
if __name__ == "__main__":
    api_key = "your_api_key_here"  # replace with your RapidAPI key

    # 1. Extract + Transform
    df_odi_topruns = fetch_top_odi_runs(api_key)
    print("Top ODI Run Scorers DataFrame:")
    print(df_odi_topruns.head())

    # 2. Load
    conn = get_connection()
    create_top_odi_runs_table(conn)
    insert_top_odi_runs(conn, df_odi_topruns)
    conn.close()

    print("🎉 Top ODI Run Scorers ETL completed successfully.")


Top ODI Run Scorers DataFrame:
   player_id   player_name  matches  innings   runs  average
0         25     Tendulkar      463      452  18426    44.83
1        104    Sangakkara      404      380  14234    41.99
2       1413         Kohli      302      290  14181    57.88
3         38     R Ponting      375      365  13704    42.04
4        102  S Jayasuriya      445      433  13430    32.36
✅ 20 ODI records inserted/updated.
🎉 Top ODI Run Scorers ETL completed successfully.


In [8]:
#venue 
import requests
import pymysql

# ========================
# Extract
# ========================
def fetch_api(url):
    """Fetch JSON data from Cricbuzz API using RapidAPI."""
    headers = {
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com",
        "x-rapidapi-key": "0ef749c92cmshb8e26740cefe8c1p143f2bjsn5d2ede97ace6"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def extract_venue_details(venue_ids):
    """Extract venue details for given venue IDs."""
    all_venues = []
    for vid in venue_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{vid}"
        try:
            data = fetch_api(url)
        except Exception as e:
            print(f"❌ Error fetching venue {vid}: {e}")
            data = {}

        venue_info = {
            "venue_id": vid,
            "venue_name": data.get("ground", "").strip() if data.get("ground") else None,
            "city": data.get("city", "").strip() if data.get("city") else None,
            "country": data.get("country", "").strip() if data.get("country") else None,
            "capacity": data.get("capacity", "").strip() if data.get("capacity") else None
        }
        all_venues.append(venue_info)

    return all_venues

# ========================
# Load
# ========================
def create_venues_table(conn):
    """Create venues table if it does not exist."""
    create_query = """
    CREATE TABLE IF NOT EXISTS venues (
        venue_id BIGINT PRIMARY KEY,
        venue_name VARCHAR(150),
        city VARCHAR(100),
        country VARCHAR(100),
        capacity VARCHAR(50)
    )
    """
    with conn.cursor() as cur:
        cur.execute(create_query)
    conn.commit()
    print("✅ Table 'venues' is ready.")

def insert_venues(conn, venues):
    """Insert venue data into MySQL, avoiding duplicates."""
    insert_query = """
    INSERT INTO venues (venue_id, venue_name, city, country, capacity)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        venue_name = VALUES(venue_name),
        city = VALUES(city),
        country = VALUES(country),
        capacity = VALUES(capacity)
    """
    with conn.cursor() as cur:
        for v in venues:
            cur.execute(insert_query, (
                v["venue_id"], v["venue_name"], v["city"], v["country"], v["capacity"]
            ))
    conn.commit()
    print(f"✅ Inserted/Updated {len(venues)} venues.")

# ========================
# ETL Pipeline
# ========================
def run_etl(venue_ids):
    print("🚀 Starting ETL pipeline...")

    # Extract
    venues = extract_venue_details(venue_ids)
    print("📥 Extracted Data:")
    for v in venues:
        print(v)

    # Load
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api"
    )

    create_venues_table(conn)
    insert_venues(conn, venues)

    conn.close()
    print("🎉 ETL pipeline completed.")


# ========================
# Run ETL
# ========================
venue_ids = [41, 34, 27, 31, 81, 19, 10, 153, 50, 40, 80, 485, 87, 11]
run_etl(venue_ids)


🚀 Starting ETL pipeline...
📥 Extracted Data:
{'venue_id': 41, 'venue_name': 'Eden Park', 'city': 'Auckland', 'country': 'New Zealand', 'capacity': '41,000'}
{'venue_id': 34, 'venue_name': 'Sydney Cricket Ground', 'city': 'Sydney', 'country': 'Australia', 'capacity': '48,000'}
{'venue_id': 27, 'venue_name': 'M.Chinnaswamy Stadium', 'city': 'Bengaluru', 'country': 'India', 'capacity': '40,000'}
{'venue_id': 31, 'venue_name': 'Eden Gardens', 'city': 'Kolkata', 'country': 'India', 'capacity': '63000'}
{'venue_id': 81, 'venue_name': 'Wankhede Stadium', 'city': 'Mumbai', 'country': 'India', 'capacity': '33,000'}
{'venue_id': 19, 'venue_name': "Lord's", 'city': 'London', 'country': 'England', 'capacity': '30000'}
{'venue_id': 10, 'venue_name': 'Brabourne Stadium', 'city': 'Mumbai', 'country': 'India', 'capacity': '20,000'}
{'venue_id': 153, 'venue_name': 'Dubai International Cricket Stadium', 'city': 'Dubai', 'country': 'United Arab Emirates', 'capacity': '25,000'}
{'venue_id': 50, 'venue_nam

In [9]:
#players_stas
import requests
import pymysql

# ----------------------
# Extract
# ----------------------
def fetch_api(url):
    headers = {
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com",
        "x-rapidapi-key": "0ef749c92cmshb8e26740cefe8c1p143f2bjsn5d2ede97ace6"   # 🔑 Replace with your key
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def extract_players_stats(player_ids):
    """
    Fetch player profile and batting stats for a list of player IDs.
    Returns a list of dicts with player_id, player_name, role, test_runs, odi_runs, t20_runs.
    """
    all_players = []

    for pid in player_ids:
        # --- Fetch profile ---
        profile_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}"
        try:
            profile_data = fetch_api(profile_url)
        except Exception as e:
            print(f"❌ Error fetching profile for {pid}: {e}")
            profile_data = {}

        player_name = profile_data.get("name", "")
        role = profile_data.get("role", "")

        # --- Fetch batting stats ---
        batting_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}/batting"
        try:
            batting_data = fetch_api(batting_url)
        except Exception as e:
            print(f"❌ Error fetching batting stats for {pid}: {e}")
            batting_data = {}

        # Initialize runs
        test_runs = odi_runs = t20_runs = 0

        values = batting_data.get("values", [])
        for item in values:
            row = item.get("values", [])
            if row and row[0] == "Runs":
                test_runs = int(row[1]) if row[1].isdigit() else 0
                odi_runs = int(row[2]) if row[2].isdigit() else 0
                t20_runs = int(row[3]) if row[3].isdigit() else 0

        all_players.append({
            "player_id": pid,
            "player_name": player_name.strip() if player_name else None,
            "role": role.strip() if role else None,
            "test_runs": test_runs,
            "odi_runs": odi_runs,
            "t20_runs": t20_runs
        })

    return all_players

# ----------------------
# Load
# ----------------------
def create_players_stats_table(conn):
    """Create players_stats table in MySQL if it doesn't exist."""
    create_query = """
    CREATE TABLE IF NOT EXISTS players_stats (
        player_id BIGINT PRIMARY KEY,
        player_name VARCHAR(100),
        role VARCHAR(50),
        test_runs INT,
        odi_runs INT,
        t20_runs INT
    )
    """
    with conn.cursor() as cur:
        cur.execute(create_query)
    conn.commit()
    print("✅ Table 'players_stats' is ready.")

def insert_players_stats(conn, players):
    """Insert or update player stats into MySQL."""
    insert_query = """
    INSERT INTO players_stats (player_id, player_name, role, test_runs, odi_runs, t20_runs)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        player_name = VALUES(player_name),
        role = VALUES(role),
        test_runs = VALUES(test_runs),
        odi_runs = VALUES(odi_runs),
        t20_runs = VALUES(t20_runs)
    """
    with conn.cursor() as cur:
        for p in players:
            cur.execute(insert_query, (
                p["player_id"],
                p["player_name"],
                p["role"],
                p["test_runs"],
                p["odi_runs"],
                p["t20_runs"]
            ))
    conn.commit()
    print(f"✅ Inserted/Updated {len(players)} players.")

# ----------------------
# ETL Pipeline
# ----------------------
def run_etl(player_ids):
    print("🚀 Starting Player Stats ETL pipeline...")

    # Extract
    players = extract_players_stats(player_ids)
    print("📥 Extracted Players:")
    for p in players:
        print(p)

    # Load
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",   # 🔑 Replace with your MySQL password
        database="cricketss_api"
    )

    create_players_stats_table(conn)
    insert_players_stats(conn, players)

    conn.close()
    print("🎉 ETL pipeline completed.")

# ----------------------
# Run
# ----------------------
if __name__ == "__main__":
    player_ids = [576, 1413, 265, 247, 2250, 8019, 6326]  # Example IDs
    run_etl(player_ids)


🚀 Starting Player Stats ETL pipeline...
📥 Extracted Players:
{'player_id': 576, 'player_name': 'Rohit Sharma', 'role': 'Batsman', 'test_runs': 4301, 'odi_runs': 11168, 't20_runs': 4231}
{'player_id': 1413, 'player_name': 'Virat Kohli', 'role': 'Batsman', 'test_runs': 9230, 'odi_runs': 14181, 't20_runs': 4188}
{'player_id': 265, 'player_name': 'MS Dhoni', 'role': 'WK-Batsman', 'test_runs': 4876, 'odi_runs': 10773, 't20_runs': 1617}
{'player_id': 247, 'player_name': 'Chris Gayle', 'role': 'Batsman', 'test_runs': 7215, 'odi_runs': 10480, 't20_runs': 1899}
{'player_id': 2250, 'player_name': 'Steven Smith', 'role': 'Batsman', 'test_runs': 10477, 'odi_runs': 5800, 't20_runs': 1094}
{'player_id': 8019, 'player_name': 'Joe Root', 'role': 'Batsman', 'test_runs': 13543, 'odi_runs': 7301, 't20_runs': 893}
{'player_id': 6326, 'player_name': 'Kane Williamson', 'role': 'Batsman', 'test_runs': 9276, 'odi_runs': 7236, 't20_runs': 2575}
✅ Table 'players_stats' is ready.
✅ Inserted/Updated 7 players.
🎉 

In [6]:
import requests
import pymysql
from datetime import datetime

# ----------------------
# STEP 1: Fetch API Data
# ----------------------
def fetch_matches_api(url, headers=None):
    if headers is None:
        headers = {
            "x-rapidapi-key": "f8e028354cmshcd0abfbb36072e8p1e4bf8jsn5f1f1d9dfc8f",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# ----------------------
# STEP 2: Extract Match Data
# ----------------------
def extract_combined_match_data(match_ids):
    all_matches = []

    for mid in match_ids:
        try:
            url_match = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}"
            url_leanback = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/leanback"

            match_data = fetch_matches_api(url_match)
            leanback_data = fetch_matches_api(url_leanback)

            match_id = match_data.get("matchid")
            series_name = match_data.get("seriesname")
            match_format = match_data.get("matchformat")

            match_date = None
            if match_data.get("startdate"):
                match_date = datetime.fromtimestamp(match_data["startdate"] / 1000).date()

            venue = match_data.get("venueinfo", {}).get("ground")
            team1 = match_data.get("team1", {}).get("teamname")
            team2 = match_data.get("team2", {}).get("teamname")

            toss_info = leanback_data.get("matchheaders", {}).get("tossresults", {})
            toss_winner = toss_info.get("tosswinnername")
            toss_decision = toss_info.get("decision")

            matchheaders = leanback_data.get("matchheaders", {})
            match_winner = None
            if matchheaders.get("status"):
                match_winner = matchheaders["status"].split(" won")[0]

            match_info = {
                "match_id": match_id,
                "format": match_format,
                "series_name": series_name,
                "match_date": str(match_date) if match_date else None,
                "venue": venue,
                "team1": team1,
                "team2": team2,
                "toss_winner": toss_winner,
                "toss_decision": toss_decision,
                "match_winner": match_winner,
                "win_margin": matchheaders.get("status")
            }

            all_matches.append(match_info)

        except Exception as e:
            print(f"❌ Failed to fetch match {mid}: {e}")

    return all_matches

# ----------------------
# STEP 3: Create Table
# ----------------------
def create_combined_matches_table(conn):
    create_query = """
    CREATE TABLE IF NOT EXISTS combined_matches (
        match_id BIGINT PRIMARY KEY,
        format VARCHAR(20),
        series_name VARCHAR(255),
        match_date DATE,
        venue VARCHAR(255),
        team1 VARCHAR(100),
        team2 VARCHAR(100),
        toss_winner VARCHAR(100),
        toss_decision VARCHAR(20),
        match_winner VARCHAR(100),
        win_margin VARCHAR(255)
    )
    """
    with conn.cursor() as cur:
        cur.execute(create_query)
    conn.commit()
    print("✅ Table 'combined_matches' is ready.")

# ----------------------
# STEP 4: Insert Data
# ----------------------
def insert_combined_matches(conn, matches):
    insert_query = """
    INSERT INTO combined_matches (
        match_id, format, series_name, match_date, venue,
        team1, team2, toss_winner, toss_decision,
        match_winner, win_margin
    ) VALUES (
        %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
    )
    ON DUPLICATE KEY UPDATE
        format=VALUES(format),
        series_name=VALUES(series_name),
        match_date=VALUES(match_date),
        venue=VALUES(venue),
        team1=VALUES(team1),
        team2=VALUES(team2),
        toss_winner=VALUES(toss_winner),
        toss_decision=VALUES(toss_decision),
        match_winner=VALUES(match_winner),
        win_margin=VALUES(win_margin)
    """
    with conn.cursor() as cur:
        for m in matches:
            cur.execute(insert_query, (
                m["match_id"], m["format"], m["series_name"], m["match_date"], m["venue"],
                m["team1"], m["team2"], m["toss_winner"], m["toss_decision"],
                m["match_winner"], m["win_margin"]
            ))
    conn.commit()
    print(f"✅ Inserted/Updated {len(matches)} combined_matches.")

# ----------------------
# STEP 5: ETL Pipeline
# ----------------------
def run_etl(match_ids):
    print("🚀 Starting Matches ETL pipeline...")

    # Extract
    matches = extract_combined_match_data(match_ids)
    print(f"📥 Extracted {len(matches)} combined_matches")

    # Load
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",   # 🔑 Replace with your MySQL password
        database="cricketss_api"
    )

    create_combined_matches_table(conn)
    insert_combined_matches(conn, matches)

    conn.close()
    print("🎉 ETL pipeline completed.")

# ----------------------
# Run
# ----------------------
if __name__ == "__main__":
    API_KEY = "f8e028354cmshcd0abfbb36072e8p1e4bf8jsn5f1f1d9dfc8f" 
    match_ids = [
        100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804,
        87626, 87878, 116972, 69862, 43061, 76535, 100312, 100321,
        112462, 60035, 78558, 60042, 100328, 75476, 59979, 59981,
        30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762
    ]
    run_etl(match_ids)


🚀 Starting Matches ETL pipeline...
📥 Extracted 32 combined_matches
✅ Table 'combined_matches' is ready.
✅ Inserted/Updated 32 combined_matches.
🎉 ETL pipeline completed.


In [13]:
#batting data
import requests
import pymysql

API_KEY = "d068e2e22dmsh6775d75453f9de9p18a87ajsn85d4fefbe7e2"

# ----------------------
# STEP 1: Fetch API
# ----------------------
def fetch_matches_api(url, headers=None):
    if headers is None:
        headers = {
            "x-rapidapi-key": API_KEY,
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# ----------------------
# STEP 2: Extract Batting Data
# ----------------------
def extract_batting_data(match_ids):
    all_records = []

    for mid in match_ids:
        try:
            url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            data = fetch_matches_api(url)

            for innings in data.get("scorecard", []):
                team_name = innings.get("batteamname")
                batsmen = innings.get("batsman", [])

                for bat in batsmen:
                    record = {
                        "match_id": mid,
                        "innings_no": innings.get("inningsid"),
                        "team": team_name,
                        "player_id": bat.get("id"),
                        "player_name": bat.get("name"),
                        "runs": int(bat.get("runs", 0)),
                        "balls_faced": int(bat.get("balls", 0)),
                        "fours": int(bat.get("fours", 0)),
                        "sixes": int(bat.get("sixes", 0)),
                        "strike_rate": float(bat.get("strkrate", 0)),
                        "dismissal": bat.get("outdec")
                    }
                    all_records.append(record)
        except Exception as e:
            print(f"❌ Failed to fetch batting data for match {mid}: {e}")

    return all_records

# ----------------------
# STEP 3: Create Table
# ----------------------
def create_batting_table(conn):
    create_query = """
    CREATE TABLE IF NOT EXISTS batting_data (
        match_id BIGINT,
        innings_no INT,
        team VARCHAR(100),
        player_id BIGINT,
        player_name VARCHAR(100),
        runs INT,
        balls_faced INT,
        fours INT,
        sixes INT,
        strike_rate FLOAT,
        dismissal VARCHAR(200)
    )
    """
    with conn.cursor() as cur:
        cur.execute(create_query)
    conn.commit()
    print("✅ Table 'batting_data' is ready.")

# ----------------------
# STEP 4: Insert Data
# ----------------------
def insert_batting_data(conn, records):
    insert_query = """
    INSERT INTO batting_data (
        match_id, innings_no, team, player_id, player_name,
        runs, balls_faced, fours, sixes, strike_rate, dismissal
    ) VALUES (
        %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
    )
    """
    with conn.cursor() as cur:
        for r in records:
            cur.execute(insert_query, (
                r["match_id"], r["innings_no"], r["team"], r["player_id"], r["player_name"],
                r["runs"], r["balls_faced"], r["fours"], r["sixes"],
                r["strike_rate"], r["dismissal"]
            ))
    conn.commit()
    print(f"✅ Inserted {len(records)} batting records.")

# ----------------------
# STEP 5: ETL Pipeline
# ----------------------
def run_batting_etl(match_ids):
    print("🚀 Starting Batting ETL pipeline...")

    # Extract
    batting_records = extract_batting_data(match_ids)
    print(f"📥 Extracted {len(batting_records)} batting rows")

    # Load
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",  # 🔑 replace with your MySQL password
        database="cricketss_api"
    )

    create_batting_table(conn)
    insert_batting_data(conn, batting_records)

    conn.close()
    print("🎉 Batting ETL completed.")

# ----------------------
# Run
# ----------------------
if __name__ == "__main__":
    match_ids = [
        75476, 100283, 100290, 130019, 82462, 116981, 116990,
        76465, 105804, 87626, 87878, 116972, 69862, 43061,
        76535, 100312, 100321, 112462, 60035, 78558, 60042,
        100328, 59979, 59981, 30554, 32047, 77894, 91778,
        91787, 105780, 105770, 105762
    ]
    run_batting_etl(match_ids)


🚀 Starting Batting ETL pipeline...
📥 Extracted 869 batting rows
✅ Table 'batting_data' is ready.
✅ Inserted 869 batting records.
🎉 Batting ETL completed.


In [ ]:
#blowing data
import requests
import pymysql

# -------------------------
# 1. API Fetch Function
# -------------------------
def fetch_matches_api(url, api_key):
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API Error {response.status_code}: {response.text}")

# -------------------------
# 2. Extract Bowling Data
# -------------------------
def extract_bowling_data(match_ids, api_key):
    all_records = []

    for mid in match_ids:
        try:
            url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            data = fetch_matches_api(url, api_key)

            for innings in data.get("scorecard", []):
                team_name = innings.get("batteamname", "Unknown")
                bowlers = innings.get("bowler", [])

                for bowler in bowlers:
                    all_records.append((
                        mid,
                        bowler.get("id"),
                        bowler.get("name"),
                        team_name,
                        float(bowler.get("overs", 0)),
                        int(bowler.get("maidens", 0)),
                        int(bowler.get("runs", 0)),
                        int(bowler.get("wickets", 0)),
                        float(bowler.get("economy", 0))
                    ))

        except Exception as e:
            print(f"Failed to fetch match {mid}: {e}")

    return all_records

# -------------------------
# 3. Create MySQL Table
# -------------------------
def create_bowling_table(connection):
    create_query = """
    CREATE TABLE IF NOT EXISTS bowling_data (
        match_id BIGINT,
        player_id BIGINT,
        player_name VARCHAR(100),
        team VARCHAR(100),
        overs FLOAT,
        maidens INT,
        runs_conceded INT,
        wickets INT,
        economy_rate FLOAT
    )
    """
    with connection.cursor() as cursor:
        cursor.execute(create_query)
    connection.commit()
    print("Table 'bowling_data' is ready.")

# -------------------------
# 4. Insert Data into MySQL
# -------------------------
def insert_bowling_data(connection, records):
    insert_query = """
    INSERT INTO bowling_data
    (match_id, player_id, player_name, team, overs, maidens, runs_conceded, wickets, economy_rate)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    with connection.cursor() as cursor:
        cursor.executemany(insert_query, records)
    connection.commit()
    print(f"Inserted {len(records)} records into 'bowling_data'.")

# =========================
# 5. Run ETL
# =========================
if __name__ == "__main__":
    api_key = "d068e2e22dmsh6775d75453f9de9p18a87ajsn85d4fefbe7e2"
    match_ids = [75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762]

    # Extract
    records = extract_bowling_data(match_ids, api_key)
    print("Extracted records:", records[:5])

    # Connect to MySQL
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='Oviya@0108',
        database='cricketss_api',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

    # Load
    create_bowling_table(connection)
    insert_bowling_data(connection, records)
    connection.close()


Extracted records: [(75476, 9311, 'Bumrah', 'Pakistan', 7.0, 1, 19, 2, 2.7), (75476, 10808, 'Siraj', 'Pakistan', 8.0, 0, 50, 2, 6.2), (75476, 9647, 'Hardik Pandya', 'Pakistan', 6.0, 0, 34, 2, 5.7), (75476, 8292, 'Kuldeep Yadav', 'Pakistan', 10.0, 0, 35, 2, 3.5), (75476, 587, 'Ravindra Jadeja', 'Pakistan', 9.5, 0, 38, 2, 3.9)]
Table 'bowling_data' is ready.
Inserted 440 records into 'bowling_data'.


In [7]:
import requests
import re
import pymysql

# -------------------------
# 1. API Fetch Function
# -------------------------
api_key = "f8e028354cmshcd0abfbb36072e8p1e4bf8jsn5f1f1d9dfc8f"

def fetch_api(url):
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None

# -------------------------
# 2. Extract Fielding Data
# -------------------------
def extract_fielding_data(match_ids):
    all_fielding = []

    for mid in match_ids:
        try:
            url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            data = fetch_api(url)
            if not data:
                continue

            # Map player names to IDs
            player_name_to_id = {}
            for innings in data.get("scorecard", []):
                for p in innings.get("batsman", []) + innings.get("bowler", []):
                    name = p.get("name")
                    if name:
                        player_name_to_id[name] = p.get("id")

            # Track fielding stats
            fielding_stats = {}
            for innings in data.get("scorecard", []):
                for bat in innings.get("batsman", []):
                    out_desc = bat.get("outdec", "")

                    # Catches
                    catch_match = re.search(r"c\s+([a-zA-Z\s]+)\s+b\s+", out_desc)
                    if not catch_match:
                        catch_match = re.search(r"c\s+([a-zA-Z\s]+)\)$", out_desc)
                    if catch_match:
                        fielder = catch_match.group(1).strip()
                        if fielder not in fielding_stats:
                            fielding_stats[fielder] = {"catches": 0, "stumpings": 0}
                        fielding_stats[fielder]["catches"] += 1

                    # Stumpings
                    stump_match = re.search(r"st\s+([a-zA-Z\s]+)\s+b\s+", out_desc)
                    if stump_match:
                        keeper = stump_match.group(1).strip()
                        if keeper not in fielding_stats:
                            fielding_stats[keeper] = {"catches": 0, "stumpings": 0}
                        fielding_stats[keeper]["stumpings"] += 1

            # Convert to list of tuples
            for pname, stats in fielding_stats.items():
                pid = player_name_to_id.get(pname)
                if pid and (stats["catches"] > 0 or stats["stumpings"] > 0):
                    all_fielding.append((
                        mid,
                        pid,
                        stats["catches"],
                        stats["stumpings"]
                    ))

        except Exception as e:
            print(f"Failed for match {mid}: {e}")

    return all_fielding

# -------------------------
# 3. Drop + Create MySQL Table
# -------------------------
def reset_fielding_table(connection):
    drop_query = "DROP TABLE IF EXISTS fielding_data"
    create_query = """
    CREATE TABLE fielding_data (
        performance_id BIGINT NOT NULL AUTO_INCREMENT,
        match_id BIGINT,
        player_id BIGINT,
        catches INT,
        stumpings INT,
        PRIMARY KEY (performance_id)
    )
    """
    with connection.cursor() as cursor:
        cursor.execute(drop_query)
        cursor.execute(create_query)
    connection.commit()
    print("Table 'fielding_data' dropped & recreated.")

# -------------------------
# 4. Insert Data into MySQL
# -------------------------
def insert_fielding_data(connection, records):
    insert_query = """
    INSERT INTO fielding_data
    (match_id, player_id, catches, stumpings)
    VALUES (%s, %s, %s, %s)
    """
    with connection.cursor() as cursor:
        cursor.executemany(insert_query, records)
    connection.commit()
    print(f"Inserted {len(records)} records into 'fielding_data'.")

# =========================
# 5. Run ETL
# =========================
if __name__ == "__main__":
    match_ids = [75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465, 105804, 87626, 87878, 116972,
              69862, 43061, 76535, 100312, 100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979, 
                  59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770, 105762]  

    # Extract
    records = extract_fielding_data(match_ids)
    print(f"Extracted {len(records)} records")

    # Connect
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='Oviya@0108',
        database='cricketss_api',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

    # Drop + Recreate table
    reset_fielding_table(connection)

    # Load
    if records:
        insert_fielding_data(connection, records)

    connection.close()


Extracted 245 records
Table 'fielding_data' dropped & recreated.
Inserted 245 records into 'fielding_data'.


In [ ]:
import requests
import pymysql
import time
from datetime import datetime

# -------------------------
# 1. API Fetch Function
# -------------------------
api_key = "4c78fe9d51msh3442ef83e518a7fp1db088jsna5390947b494"

def fetch_matches_api(url):
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"API Error: {e}")
        return None

# -------------------------
# 2. Extract Batting Data
# -------------------------
def extract_batting_data(match_ids, pause=1):
    records = []
    for match_id in match_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/hscard"
        data = fetch_matches_api(url)

        if not data:
            continue

        # Match date
        match_date = None
        match_date_ms = data.get("responselastupdated")
        if match_date_ms:
            match_date = datetime.fromtimestamp(match_date_ms / 1000).date()

        # Loop through innings
        for innings in data.get("scorecard", []):
            innings_no = innings.get("inningsid")
            team_name = innings.get("batteamname")

            for batsman in innings.get("batsman", []):
                record = (
                    match_id,
                    innings_no,
                    batsman.get("id"),
                    batsman.get("name"),
                    int(batsman.get("runs", 0)),
                    int(batsman.get("balls", 0)),
                    float(batsman.get("strkrate", 0)),
                    str(match_date)
                )
                records.append(record)

        # Sleep to avoid hitting rate limit
        time.sleep(pause)

    return records

# -------------------------
# 3. Reset MySQL Table
# -------------------------
def reset_batting_table(connection):
    drop_query = "DROP TABLE IF EXISTS batters_batting_data"

    create_query = """
    CREATE TABLE batters_batting_data (
        performance_id BIGINT NOT NULL AUTO_INCREMENT,
        match_id BIGINT,
        innings_no INT,
        player_id BIGINT,
        player_name VARCHAR(255),
        runs INT,
        balls_faced INT,
        strike_rate FLOAT,
        match_date DATE,
        PRIMARY KEY (performance_id),
        UNIQUE KEY unique_record (match_id, innings_no, player_id)
    )
    """
    with connection.cursor() as cursor:
        cursor.execute(drop_query)
        cursor.execute(create_query)
    connection.commit()
    print("Table 'batters_batting_data' dropped & recreated.")

# -------------------------
# 4. Insert Data into MySQL
# -------------------------
def insert_batting_data(connection, records):
    insert_query = """
    INSERT IGNORE INTO batters_batting_data
    (match_id, innings_no, player_id, player_name, runs, balls_faced, strike_rate, match_date)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """
    with connection.cursor() as cursor:
        cursor.executemany(insert_query, records)
    connection.commit()
    print(f"Inserted {len(records)} batting records.")

# -------------------------
# 5. Run ETL
# -------------------------
if __name__ == "__main__":
    match_ids = [
       75476, 100283, 100290, 130019, 82462, 116981, 116990, 76465,
       105804, 87626, 87878, 116972, 69862, 43061, 76535, 100312,
       100321, 112462, 60035, 78558, 60042, 100328, 75476, 59979,
       59981, 30554, 32047, 77894, 91778, 91787, 105780, 105770,
       105762
    ] 

    # Extract
    records = extract_batting_data(match_ids, pause=1)
    print(f"Extracted {len(records)} batting records")

    # Connect to MySQL
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='Oviya@0108',
        database='cricketss_api',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

    # Reset table & load data
    reset_batting_table(connection)
    if records:
        insert_batting_data(connection, records)

    connection.close()


Extracted 891 batting records
Table 'batters_batting_data' dropped & recreated.
Inserted 891 batting records.


In [4]:
import requests
import pymysql
import time

# ---------------- API Helper ----------------
api_key = "e6f772e53cmsh35b97b5b358e839p17d820jsn89d15a173062"

def fetch_api(url):
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"API error for {url}: {e}")
        return None

# ---------------- Extract & Transform ----------------
def extract_bowling_data(match_ids, pause=1):
    records = []

    for mid in match_ids:
        url_scorecard = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
        url_match = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}"

        scorecard = fetch_api(url_scorecard)
        match = fetch_api(url_match)

        if not scorecard or not match:
            continue

        # Venue (ground + city)
        venue_info = match.get("venueinfo", {})
        venue = f"{venue_info.get('ground', '')}, {venue_info.get('city', '')}".strip(", ")

        match_id = match.get("matchid", mid)
        for innings in scorecard.get("scorecard", []):
            innings_no = innings.get("inningsid", 0)
            for bowler in innings.get("bowler", []):
                record = (
                    match_id,
                    innings_no,
                    bowler.get("id", 0),
                    bowler.get("name", ""),
                    float(bowler.get("overs", 0)),
                    int(bowler.get("runs", 0)),
                    int(bowler.get("wickets", 0)),
                    float(bowler.get("economy", 0)),
                    venue
                )
                records.append(record)

        time.sleep(pause)  # avoid rate limit

    return records

# ---------------- Load (MySQL) ----------------
def reset_bowling_table(conn):
    drop_query = "DROP TABLE IF EXISTS bowlers_bowling_venue_data"
    create_query = """
    CREATE TABLE bowlers_bowling_venue_data (
        performance_id BIGINT NOT NULL AUTO_INCREMENT,
        match_id BIGINT,
        innings_no INT,
        player_id BIGINT,
        player_name VARCHAR(100),
        overs FLOAT,
        runs_conceded INT,
        wickets INT,
        economy_rate FLOAT,
        venue VARCHAR(200),
        PRIMARY KEY (performance_id)
    )
    """
    with conn.cursor() as cursor:
        cursor.execute(drop_query)
        cursor.execute(create_query)
    conn.commit()
    print("Table 'bowlers_bowling_venue_data' recreated.")

def insert_bowling_data(conn, records):
    insert_query = """
    INSERT INTO bowlers_bowling_venue_data
    (match_id, innings_no, player_id, player_name, overs, runs_conceded, wickets, economy_rate, venue)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    with conn.cursor() as cursor:
        cursor.executemany(insert_query, records)
    conn.commit()
    print(f"Inserted {len(records)} bowling records.")

# ---------------- Run ETL ----------------
if __name__ == "__main__":
    match_ids = [100283, 38617, 38612, 38627, 75623, 78621, 48121, 70326, 91787, 60035, 100310, 75563]

    # Extract
    records = extract_bowling_data(match_ids, pause=1)
    print(f"Extracted {len(records)} bowling rows")

    # Connect MySQL
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api",
        charset="utf8mb4"
    )

    # Load
    reset_bowling_table(conn)
    if records:
        insert_bowling_data(conn, records)

    conn.close()


Extracted 174 bowling rows
Table 'bowlers_bowling_venue_data' recreated.
Inserted 174 bowling records.


In [ ]:
import requests
import pymysql
import time
import random

# =========================
# 1. Extract
# =========================
def fetch_match_api(url, headers, retries=3, delay=2):
    """
    Fetch data from API with retry mechanism to handle rate limits.
    """
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Too Many Requests
                wait_time = delay * (attempt + 1) + random.uniform(1, 3)
                print(f"Rate limit hit. Retrying after {wait_time:.1f} seconds...")
                time.sleep(wait_time)
        except Exception as e:
            print(f"API fetch failed: {e}")
    return None


def extract_partnerships_data(match_ids, api_key):
    """
    Extracts partnerships data for multiple matches.
    """
    all_partnerships = []

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    for mid in match_ids:
        try:
            url_scorecard = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            scorecard_data = fetch_match_api(url_scorecard, headers)

            if not scorecard_data:
                continue

            score_cards = scorecard_data.get("scorecard", [])
            
            for innings in score_cards:
                innings_no = innings.get("inningsid", 0)
                partnerships_list = innings.get("partnership", {}).get("partnership", [])

                for index, partnership_info in enumerate(partnerships_list):
                    record = {
                        "match_id": mid,
                        "innings_no": innings_no,
                        "batter1_id": partnership_info.get("bat1id", 0),
                        "batter2_id": partnership_info.get("bat2id", 0),
                        "batter1_name": partnership_info.get("bat1name", ""),
                        "batter2_name": partnership_info.get("bat2name", ""),
                        "runs_partnership": partnership_info.get("totalruns", 0),
                        "balls_faced": partnership_info.get("totalballs", 0),
                        "wicket_fallen": index + 1
                    }
                    all_partnerships.append(record)

            # Random short sleep between matches to reduce API load
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Failed to process match {mid}: {e}")

    return all_partnerships


# =========================
# 2. Transform
# =========================
def transform_partnerships(records):
    """
    Transform raw extracted data if needed (cleaning, typecasting).
    """
    transformed = []
    for rec in records:
        transformed.append((
            int(rec["match_id"]),
            int(rec["innings_no"]),
            int(rec["batter1_id"]),
            int(rec["batter2_id"]),
            rec["batter1_name"].strip(),
            rec["batter2_name"].strip(),
            int(rec["runs_partnership"]),
            int(rec["balls_faced"]),
            int(rec["wicket_fallen"])
        ))
    return transformed


# =========================
# 3. Load
# =========================
def create_partnerships_table(conn):
    """
    Creates 'players_partnerships_data' table if not exists.
    """
    query = """
    CREATE TABLE IF NOT EXISTS players_partnerships_data (
        match_id BIGINT,
        innings_no INT,
        batter1_id BIGINT,
        batter2_id BIGINT,
        batter1_name VARCHAR(100),
        batter2_name VARCHAR(100),
        runs_partnership INT,
        balls_faced INT,
        wicket_fallen INT
    )
    """
    with conn.cursor() as cur:
        cur.execute(query)
    conn.commit()


def load_players_partnerships_data(conn, records):
    """
    Insert transformed data into MySQL table.
    """
    insert_query = """
    INSERT INTO players_partnerships_data
    (match_id, innings_no, batter1_id, batter2_id, batter1_name, batter2_name, runs_partnership, balls_faced, wicket_fallen)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    with conn.cursor() as cur:
        cur.executemany(insert_query, records)
    conn.commit()
    print(f"Inserted {len(records)} records into partnerships_data")


# =========================
# 4. Run ETL
# =========================
if __name__ == "__main__":
    api_key = "ef8e028354cmshcd0abfbb36072e8p1e4bf8jsn5f1f1d9dfc8f"
    match_ids = [38321, 50958, 59965, 48205, 48401]  

    # Step 1: Extract
    extracted_data = extract_partnerships_data(match_ids, api_key)

    # Step 2: Transform
    transformed_data = transform_partnerships(extracted_data)

    # Step 3: Load
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Oviya@0108",
        database="cricketss_api"
    )

    create_partnerships_table(conn)
    load_partnerships_data(conn, transformed_data)

    conn.close()


Rate limit hit. Retrying after 6.3 seconds...
Rate limit hit. Retrying after 4.8 seconds...
Rate limit hit. Retrying after 7.4 seconds...
Rate limit hit. Retrying after 6.0 seconds...
Rate limit hit. Retrying after 4.2 seconds...
Rate limit hit. Retrying after 7.6 seconds...
Rate limit hit. Retrying after 8.0 seconds...
Inserted 0 records into partnerships_data


In [ ]:
import requests
import pymysql
import pandas as pd

# ==============================
# 1. Extract
# ==============================

API_KEY = "f8e028354cmshcd0abfbb36072e8p1e4bf8jsn5f1f1d9dfc8f"
API_HOST = "cricbuzz-cricket.p.rapidapi.com"

def fetch_api(url):
    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": API_HOST
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None
    except Exception as e:
        print(f"API Fetch Error: {e}")
        return None


def extract_partnerships_data(match_ids):
    all_partnerships = []

    for mid in match_ids:
        try:
            url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{mid}/hscard"
            data = fetch_api(url)

            if not data:
                continue

            score_cards = data.get("scorecard", [])
            for innings in score_cards:
                innings_no = innings.get("inningsid", 0)
                partnerships_list = innings.get("partnership", {}).get("partnership", [])

                for index, partnership in enumerate(partnerships_list):
                    record = {
                        "match_id": mid,
                        "innings_no": innings_no,
                        "batter1_id": partnership.get("bat1id", 0),
                        "batter2_id": partnership.get("bat2id", 0),
                        "batter1_name": partnership.get("bat1name", ""),
                        "batter2_name": partnership.get("bat2name", ""),
                        "runs_partnership": partnership.get("totalruns", 0),
                        "balls_faced": partnership.get("totalballs", 0),
                        "wicket_fallen": index + 1
                    }
                    all_partnerships.append(record)

        except Exception as e:
            print(f"Failed for match {mid}: {e}")

    return pd.DataFrame(all_partnerships)


# ==============================
# 2. Load
# ==============================

def create_connection(host, user, password, db):
    try:
        conn = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=db,
            autocommit=True
        )
        return conn
    except Exception as e:
        print(f"MySQL Connection Error: {e}")
        return None


def create_table(conn):
    query = """
    CREATE TABLE IF NOT EXISTS players_partnerships_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        match_id INT,
        innings_no INT,
        batter1_id INT,
        batter2_id INT,
        batter1_name VARCHAR(100),
        batter2_name VARCHAR(100),
        runs_partnership INT,
        balls_faced INT,
        wicket_fallen INT
    )
    """
    with conn.cursor() as cur:
        cur.execute(query)
    print("✅ Table 'players_partnerships_data' ready.")


def insert_partnerships(df, conn):
    insert_query = """
    INSERT INTO players_partnerships_data
    (match_id, innings_no, batter1_id, batter2_id, batter1_name, batter2_name, runs_partnership, balls_faced, wicket_fallen)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    records = df.to_records(index=False)
    with conn.cursor() as cur:
        cur.executemany(insert_query, records)
    conn.commit()
    print(f"✅ Inserted {len(records)} rows into MySQL.")


# ==============================
# 3. Run ETL
# ==============================

if __name__ == "__main__":
    # Match IDs for testing
    match_ids = [38321, 50958, 59965, 48205, 48401]

    # Extract
    df_partnerships = extract_partnerships_data(match_ids)
    print(df_partnerships.head())

    # Load
    conn = create_connection("localhost", "root", "Oviya@0108", "cricketss_api")
    if conn:
        create_table(conn)
        insert_partnerships(df_partnerships, conn)
        conn.close()


   match_id  innings_no  batter1_id  batter2_id batter1_name batter2_name  \
0     38321           1        6534        6507          Roy     Bairstow   
1     38321           1        6534        8019          Roy         Root   
2     38321           1        6534        6557          Roy       Stokes   
3     38321           1        2258        6557      Buttler       Stokes   
4     38321           1        2258        6692      Buttler    Moeen Ali   

   runs_partnership  balls_faced  wicket_fallen  
0                12            9              1  
1                 0            3              2  
2                54           47              3  
3                 8           21              4  
4                75           84              5  
✅ Table 'players_partnerships_data' ready.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()